# Formula 1 data analysis application season 2022
The data I can find

This application is reusable for the futur seasons

## Imports

In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.express as px
import chart_studio.plotly as py
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import json

## Backend
In this part, I do everyting that is different from the graphs and charts display.
It allows me to dirctly have all the data and process them so I just hav to select the graphs after that 

In [28]:
# import data
data_all = pd.read_csv("data.csv", sep=",")
data_all["index"] = data_all.groupby("pilot").cumcount()
data_all["index"] += 1

# dataset of the cumulated points by pilot
data_points = data_all[["pilot", "points"]].copy()
data_points["cum_points"] = data_points.groupby("pilot").cumsum()
data_points["index"] = data_points.groupby("pilot").cumcount()

# extraction of the pilots by team
with open("teams.json") as jsonfile:
    teams_pilot = json.load(jsonfile)
data_team_points = {}
for team in teams_pilot.keys():
    data_team_points[team] = []
    
# for each team we sum the points of each pilot of the team
for team in teams_pilot.keys():
    s = 0
    j = 0
    for i in range(data_points.shape[0]):
        if data_points.iloc[i]["pilot"] in teams_pilot[team]:
            s += data_points.iloc[i]["points"]
            j += 1
        if j == 2:
            j = 0
            data_team_points[team].append(s)
data_team_points = pd.DataFrame(data_team_points)
test = []
for i in range(data_team_points.shape[0]):
    for team in teams_pilot.keys():
        test.append([team, data_team_points.iloc[i][team]])
data_team_points = pd.DataFrame(test, columns=["team", "points"])
data_team_points["index"] = data_team_points.groupby("team").cumcount()

## Frontend
Here the displays will be separated to choose what we display

### Test data import

In [19]:
print(data_all.head())

  pilot  avg_speed  best_lap  grid  points  result      time  Unnamed: 7  \
0   lec    206.018    94.570     1      33       1  5853.584         NaN   
1   sai    203.501    95.740     3      23       2  5859.182         NaN   
2   ham    202.469    96.228     5      15       3  5863.259         NaN   
3   rus    202.313    96.302     9      12       4  5864.795         NaN   
4   mag    201.641    96.623     7      11       5  5868.338         NaN   

   index  
0      1  
1      1  
2      1  
3      1  
4      1  


### Scattered Points
A graph with grid position on the x axis, and the result in the y axis. It allows me to predict if the order of the grid is kept

In [20]:
px.scatter(data_all, x="grid", y="result", color="pilot", title="Result/Grid", template="plotly_dark")

Still scattered points, but with the total time in x axis and best lap time in y axis

In [21]:
px.scatter(data_all, x="time", y="best_lap", color="pilot", title="Best time per pilot", template="plotly_dark", size="index")

### Heatmap

Here I'll do all the heatmaps

The first one compare grid and result

In [22]:
px.density_heatmap(data_all, x="grid", y="result", title="Result/Grid heatmap", template="plotly_dark")

### Evolutions

Here its the curves of evolution along the season

In [29]:
px.line(data_points, x="index", y="cum_points", color="pilot", title="Pilots cumlated points", template="plotly_dark", markers=True)

Same thing but with teams

In [30]:
px.line(data_team_points, x="index", y="points", color="team", title="Teams cumulated points", template="plotly_dark", markers=True)

As the main battle this year is between Perez and Leclerc,  little chart to get the points difference between these two pilots :

In [46]:
# points difference between per and lec
diff = []
col = []
for i in range(22):
    per = int(data_points["cum_points"][data_points["pilot"] == "per"][data_points["index"] == i])
    lec = int(data_points["cum_points"][data_points["pilot"] == "lec"][data_points["index"] == i])
    diff.append(int(per-lec))
    if int(per-lec) > 0:
        col.append("green")
    else:
        col.append("red")
px.bar(x=range(1, len(diff)+1), y=diff, title="Points difference between Perez and Leclerc", template="plotly_dark", color=col, color_discrete_map={
                "red": "green",
                "green": "red"})

## Conclusion

After this year, and this incredible season, I have to do some conclusion to end correctly.

### About the experiment

It was full of things to learn on data visualisation, extract data, using some technologies like plotly that I never used before. After this year I want only one thing, and it's to continue

### About F1

In the data, we can clearly see that despite some problems on the beginning of the season, Verstappen really nailed it. Leclerc on the other hand started the season perfectly but had so technical issues, strategy issues and pilot errors.
Mercedes started in the middle of the grid, and finally got very close to the two top teams
While Norris was the best of the rest, Ricciardo really struggled with te car. Their main competitor is Alpine. They had a lot of issues with the fiability, especially Fernando Alonso, who sometimes was almost has quick has the top teams
Concerning the rest of the grid, it's more difficult to say.

### What's next

Since the Brazilian GP, I started to plan the next season. I really want to do something better next year. To Be Continued